In [3]:
# Download data from http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [4]:
import numpy as np
import re
import collections
import operator
from scipy import optimize as opt

In [5]:
# Read data
negDataPath = !ls data/neg/
posDataPath = !ls data/pos/

# Read only 1000 neg and 1000 pos data
N = 200
negDataPath = negDataPath[0:N/2]
posDataPath = posDataPath[0:N/2]

negData = []
posData = []

for i in negDataPath:
    txt = open('./data/neg/' + i).read()
    negData.append(txt)
    
for i in posDataPath:
    txt = open('./data/pos/' + i).read()
    posData.append(txt)

In [6]:
# Find words count
negDataDic = np.array([])
posDataDic = np.array([])
allWords = {}

for review in negData:
    words = re.compile('[a-zA-Z\']*[a-zA-Z]').findall(review)
    wordsDic = collections.Counter(words)
    negDataDic = np.append(negDataDic, wordsDic)
    allWords.update(wordsDic)
    
for review in posData:
    words = re.compile('[a-zA-Z\']*[a-zA-Z]').findall(review)
    wordsDic = collections.Counter(words)
    posDataDic = np.append(posDataDic, wordsDic)
    allWords.update(wordsDic)

In [7]:
index = 0
for key in allWords:
    allWords[key] = index * 1
    index+=1

In [8]:
def findRow(allWords, text):
    row = np.zeros(len(allWords))
    words = re.compile('[a-zA-Z\']*[a-zA-Z]').findall(text)
    wordsDic = collections.Counter(words)
    for word in words:
        if word in allWords:
            row[allWords[word]] = wordsDic[word]
    return row

In [9]:
x = []

for i in range(N/2):
    x.append( findRow(allWords, negData[i]) )
    
for i in range(N/2):
    x.append( findRow(allWords, posData[i]) )

x = np.row_stack(x)

y = np.append(-1 * np.ones( (1, N/2) ), np.ones( (1, N/2) ))

In [10]:
# Fix 💩
def cost(w, b, c, x, y):
    p = 1 - y * (x.dot(w) + b)
    p[p<0] = 0
    p = p**2
    return w.dot(w.T) + c * p.sum()

In [11]:
M = len(allWords)
w = np.zeros(M)
b = 0
c = 100

cost(w, b, c, x, y)

20000.0

In [12]:
M = len(allWords)
w = np.zeros(M)
b = 0
c = 100

res = opt.minimize(cost, w, method='nelder-mead', args=(b, c, x, y), options={'xtol': 1e-8, 'disp': True, 'maxiter': 10})
res.x

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])